<a href="https://colab.research.google.com/github/varunika09/Excelr-assignments/blob/main/Homeowrk%208/Homework8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install newspaper3k sentence-transformers vaderSentiment nrclex keybert transformers torch lxml_html_clean --quiet


In [7]:
from newspaper import Article
from sentence_transformers import SentenceTransformer, util
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nrclex import NRCLex
from keybert import KeyBERT
from transformers import pipeline


In [4]:
url2 = "https://www.foxweather.com/weather-news/hurricane-melissa-jamaica-haiti-cuba-dominican-republic-caribbean-atlantic"
url1 = "https://www.nytimes.com/2025/10/26/weather/jamaica-worst-storms-hurricane-melissa.html"

In [9]:
def extract_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.title, article.text

title1, text1 = extract_article(url1)
title2, text2 = extract_article(url2)

print("NYT Title:", title1)
print("Fox News Title:", title2)
print("\nArticles extracted successfully.\n")

NYT Title: Hurricane Melissa Could Hit as One of Jamaica’s Strongest Storms Ever
Fox News Title: Hurricane Melissa resumes intensifying near Jamaica with dire warnings of catastrophic flooding, landslides

Articles extracted successfully.



In [19]:
print(text1)

In August 1951, Hurricane Charlie slammed into Jamaica and was considered at the time to be the nation’s “worst hurricane disaster of the century,” according to a federal report.

Charlie packed winds greater than 110 m.p.h. and brought 17 inches of rain, causing an estimated $50 million (more than $620 million today) in damage to property and crops. The storm killed 152 people, injured 2,000 others and left approximately 25,000 residents homeless.

Charlie remained the last hurricane to make landfall in Jamaica until Gilbert arrived 37 years later.

A Category 3 hurricane in 1912 hit the island with peak sustained winds of 115 m.p.h. However, in a re-analysis by the Hurricane Center, it was noted that rainfall leading to flash flooding proved to be the most destructive, especially in the northwest side of the island.

Several towns were reported to have been wiped out by the winds and tidal waves and at least 100 lives were lost in that storm.


In [11]:
#Measure similarity

model = SentenceTransformer('all-MiniLM-L6-v2')

emb1 = model.encode(text1, convert_to_tensor=True)
emb2 = model.encode(text2, convert_to_tensor=True)

similarity_score = util.pytorch_cos_sim(emb1, emb2).item()
print(f"Semantic Similarity: {similarity_score:.4f}\n")


Semantic Similarity: 0.6363



In [12]:
#Sentiment analysis

analyzer = SentimentIntensityAnalyzer()

sent1 = analyzer.polarity_scores(text1)
sent2 = analyzer.polarity_scores(text2)

print("NYT Sentiment:", sent1)
print("Fox News Sentiment:", sent2)


NYT Sentiment: {'neg': 0.119, 'neu': 0.866, 'pos': 0.014, 'compound': -0.9617}
Fox News Sentiment: {'neg': 0.095, 'neu': 0.862, 'pos': 0.043, 'compound': -0.998}


In [15]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

from nrclex import NRCLex


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [16]:
#Emotion analysis

emotion1 = NRCLex(text1)
emotion2 = NRCLex(text2)

print("\nNYT Top Emotions:", emotion1.top_emotions)
print("Fox News Top Emotions:", emotion2.top_emotions)


NYT Top Emotions: [('negative', 0.2857142857142857)]
Fox News Top Emotions: [('negative', 0.2006079027355623)]


In [17]:
#Keyword extraction

kw_model = KeyBERT(model='all-MiniLM-L6-v2')

keywords1 = kw_model.extract_keywords(text1, top_n=5)
keywords2 = kw_model.extract_keywords(text2, top_n=5)

print("\nNYT Top 5 Keywords:")
for kw in keywords1:
    print("-", kw[0])

print("\nFox News Top 5 Keywords:")
for kw in keywords2:
    print("-", kw[0])


NYT Top 5 Keywords:
- hurricane
- storm
- jamaica
- landfall
- charlie

Fox News Top 5 Keywords:
- hurricane
- jamaica
- jamaican
- kingston
- landfalling


In [18]:
#Summarization (LLM)

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

summary1 = summarizer(text1[:3000], max_length=120, min_length=60, do_sample=False)[0]['summary_text']
summary2 = summarizer(text2[:3000], max_length=120, min_length=60, do_sample=False)[0]['summary_text']

print("\n🗞 NYT Summary:\n", summary1)
print("\n🗞 Fox News Summary:\n", summary2)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu



🗞 NYT Summary:
 In August 1951, Hurricane Charlie slammed into Jamaica and was considered at the time to be the nation’s “worst hurricane disaster of the century,” according to a federal report. The storm killed 152 people, injured 2,000 others and left approximately 25,000 residents homeless. Charlie remained the last hurricane to make landfall in Jamaica until Gilbert arrived 37 years later.

🗞 Fox News Summary:
 Hurricane Melissa is centered about 115 miles south of Kingston, Jamaica. Storm has maximum sustained winds of 145 mph, making it a Category 4 hurricane. Strengthening is expected to continue, and the storm will likely become a catastrophic Category 5 hurricane as it creeps closer to Jamaica. Forecasters warn of "extensive infrastructural damage, long-duration power and communication outages"


In [21]:
#Final Findings
print("FINAL FINDINGS\n")

print(f"Similarity Score: {similarity_score:.4f}")

def sentiment_label(s):
    if s['compound'] > 0.05:
        return "Positive"
    elif s['compound'] < -0.05:
        return "Negative"
    else:
        return "Neutral"

print(f"NYT Sentiment: {sentiment_label(sent1)} ({sent1})")
print(f"Fox News Sentiment: {sentiment_label(sent2)} ({sent2})")
print("NYT Dominant Emotion:", emotion1.top_emotions[0][0] if emotion1.top_emotions else "N/A")
print("Fox News Dominant Emotion:", emotion2.top_emotions[0][0] if emotion2.top_emotions else "N/A")

print("\nNYT Keywords:", [k[0] for k in keywords1])
print("Fox News Keywords:", [k[0] for k in keywords2])

FINAL FINDINGS

Similarity Score: 0.6363
NYT Sentiment: Negative ({'neg': 0.119, 'neu': 0.866, 'pos': 0.014, 'compound': -0.9617})
Fox News Sentiment: Negative ({'neg': 0.095, 'neu': 0.862, 'pos': 0.043, 'compound': -0.998})
NYT Dominant Emotion: negative
Fox News Dominant Emotion: negative

NYT Keywords: ['hurricane', 'storm', 'jamaica', 'landfall', 'charlie']
Fox News Keywords: ['hurricane', 'jamaica', 'jamaican', 'kingston', 'landfalling']
